## 로그인

In [113]:
import requests
from bs4 import BeautifulSoup as bs

In [182]:
# 로그인할 유저정보를 넣어줍시다. (모두 문자열입니다!)
LOGIN_INFO = {
    'locale': 'ko_KR',
    'dynamicForm' : 'login',
    'templatePath': '/authentication/login',
    #'successUrl' : '/',
    'userId': '',
    'j_username': 'zzz5712',
    'j_password': '!Vkxqud1004',
    'breeze-me': 'on'
}

In [163]:
login_header = {
    'postman-token': '0a101b18-8933-4834-e4a3-63468bec2ff9',
    'cache-control': 'no-cache',
    'referer': 'https://www.nike.com/kr/ko_kr/login',
    'content-type': 'application/x-www-form-urlencoded',
    'x-requested-with': 'XMLHttpRequest'
}

In [ ]:
 headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}

In [281]:
priority_size = [260, 265, 270, 255, 275, 280, 250, 285, 290]

In [284]:
europe_size = {240 : 35, 245 : 36, 250 : 38, 255:39, 260:42, 265:43, 270:46, 275:47, 280:50, 285:51, 290:54, 295:55, 300:58, 305:59, 310:62, 320:66}

In [196]:
with requests.Session() as sess:
    # HTTP GET Request: requests대신 sess 객체를 사용한다.
   
    login_page = sess.get( url='https://www.nike.com/kr/ko_kr/login?successUrl=/', headers = headers) #한국 나이키 서버는 헤더가 없는 요청을 거부
    html = login_page.text
    soup = bs(html, 'html.parser')
    csrf = soup.find('input', {'name': 'csrfToken'})
    LOGIN_INFO = {**LOGIN_INFO, **{'csrfToken': csrf['value']}}
    print(LOGIN_INFO)


{'locale': 'ko_KR', 'dynamicForm': 'login', 'templatePath': '/authentication/login', 'userId': '', 'j_username': 'zzz5712', 'j_password': '!Vkxqud1004', 'breeze-me': 'on', 'csrfToken': 'YXGK-85SE-7Z02-XEEU-R0I1-4NA3-TAPR-VKGO'}


In [202]:
login_req = sess.post('https://www.nike.com/kr/ko_kr/login_post.htm', data=LOGIN_INFO, headers=login_header)
if login_req.status_code != 200: #status_code type is int
     raise Exception("Login Failed")

## 링크 접속

In [203]:
early_link = input('early link, please : ')

early link, please : https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/924453-012/momq00/air-vapormax-plus


In [253]:
product_req = sess.get(early_link, headers=headers)
product_soup = bs(product_req.text, 'html.parser')

## 수량확인

In [287]:
order_info = {'itemAttributes[FW_SIZE]': '', # 260
    'SIZE': '', #42
    'quantity': '1',
    'fType': 'PHYSICAL_SHIP',
    'csrfToken': '', 
    'productId':  ''} #10000008780

In [288]:
order_info['productId'] = product_soup.find('a', {'class':'review-write-btn'})['data-productid']

In [322]:
#product_DB = sess.get('https://www.nike.com/kr/ko_kr/productSkuInventory?productId='+product_soup.find('input', {'name':'productId'})['value'], headers={'Referer' : early_link})
product_DB = sess.get('https://www.nike.com/kr/ko_kr/productSkuInventory?productId='+order_info['productId'], headers={'Referer' : early_link})

product_DB = product_DB.json()

In [290]:
def select_size(product_DB = product_DB, priority_size=priority_size):
    for item in priority_size:
        number = int((int(item) - 240)/5)
        #print(item)
        #print(product_DB['skuPricing'][number]['quantity'])
        if  0 != product_DB['skuPricing'][number]['quantity']:
            return item

In [291]:
order_info['csrfToken'] = pl_soup.find('input', {'name': 'csrfToken'})
order_info['itemAttributes[FW_SIZE]'] = select_size()
order_info['SIZE'] = europe_size[select_size()]

In [353]:
order_headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    #'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'referer' : early_link,
    #'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    #'origin': 'https://www.nike.com'
}

In [354]:
order_req = sess.post('https://www.nike.com/kr/ko_kr/cart/add?directOrder=true', data=order_info, headers=order_headers)

In [355]:
order_req.status_code

500